In [ ]:
import numpy as np
import perceval as pcvl

## Avant de commencer
Voici quelques utilitaires : 

In [ ]:
from perceval import pdisplay, PS, BS, Circuit, BasicState, Processor, StateVector
from perceval.backends import BackendFactory
from perceval.algorithm import Analyzer
from exqalibur import FockState
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
from numpy import pi, cos, sin
from typing import Optional


# On reprend notre encodage par rail
qubits = {
    "0": BasicState([1, 0]),
    "1": BasicState([0, 1])
}
qubits_ = {qubits[k]: k for k in qubits}
sqlist = [qubits["0"], qubits["1"]]

# Analyse du circuit
def analyze(circuit: Circuit, input_states: Optional[FockState] = None, output_states: Optional[FockState] = None) \
        -> None:
    if input_states is None:
        input_states = sqlist
    if output_states is None:
        output_states = sqlist
    p = Processor("Naive", circuit)
    a = Analyzer(p, input_states, output_states, mapping=qubits_)
    pdisplay(a)

# Analyse du circuit en calculant les amplitudes
def amplitudes(circuit: Circuit, input_state: Optional[FockState] = None, output_states: Optional[FockState] = None) \
        -> (complex, complex):
    if input_state is None:
        input_state = qubits["0"]
    if output_states is None:
        output_states = sqlist
    b = BackendFactory.get_backend("Naive")
    b.set_circuit(circuit)
    b.set_input_state(input_state)
    return {qubits_[k]: roundc(b.prob_amplitude(k)) for k in output_states}

# Affichage de la sphère de Bloch
def circuit_to_state_vector(circuit: Circuit) -> Statevector:
    ampl0, ampl1 = amplitudes(circuit)
    return Statevector([ampl0, ampl1])
plot_bloch = lambda circuit : plot_bloch_multivector(circuit_to_state_vector(circuit))

# Rotations
x_rot = lambda x: Circuit(2) // (0, PS(pi)) // BS.Rx(theta=x) // (0, PS(pi)) 
y_rot = lambda x: BS.Ry(theta=x)
z_rot = lambda x: BS.H() // x_rot(x) // BS.H() 

# Trigonométrie avec Matplotlib 
def plot_trig(angles, colors=None, annotations=None):
    r = 1.5
    if colors is None:
        colors = ["blue"] * len(angles)
    if annotations is None:
        annotations = [""] * len(angles)
    for angle, color, annotation in zip(angles, colors, annotations):
        pos_x = r * cos(angle)
        pos_y = r * sin(angle)
        plt.plot([0, pos_x], [0, pos_y], color=color)
        pos_x_a = pos_x + np.sign(pos_x) * 0.1 - (0.05 * len(annotation) if np.sign(pos_x) < 0 else 0)
        pos_y_a = pos_y + np.sign(pos_y) * 0.1
        plt.gca().annotate(annotation, xy=(pos_x_a, pos_y_a), xycoords='data', fontsize=10)

    plt.plot(0, 0, color='black', marker='o')
    a = np.linspace(0 * pi, 2 * pi, 100)
    xs, ys = r * cos(a), r * sin(a)
    plt.plot(xs, ys, color="black")
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.gca().set_aspect('equal')
    plt.show()

# Version de `round()` pour les nombres complexes.
def roundc(c: complex, decimals: int = 2) -> complex:
    return round(c.real, decimals) + round(c.imag, decimals) * 1j

### Exemples

In [ ]:
hadamard_gate = BS.H()
print(amplitudes(hadamard_gate))
analyze(hadamard_gate)
plot_bloch(hadamard_gate)

Il n'existe pas uniquement la sphère de Bloch pour représenter des qubits, on peut simplement utiliser un cercle trigonométrique. La transition est explicite : **l'abscisse correspond à $|0\rangle$ et l'ordonnée à $|1\rangle$**, $|+\rangle$ se retrouve donc à $\frac{\pi}{4}$. Tous les états sont sur le cercle, car la somme des probabilités, qui ici est une amplitude, est toujours $1$. On perd néanmoins, avec cet affichage, l'information de la phase globale.

Par exemple, voici la base canonique (en bleu) et la base d'Hadamard (en rouge) : 

In [ ]:
plot_trig([0, pi/2, pi/4, -pi/4], ["blue", "blue", "red", "red"], ["|0>", "|1>", "|+>", "|->"])

Pour ce challenge, comme nous n'avons pas besoin de phase, mais de visualiser plusieurs bases différentes en même temps, nous allons utiliser uniquement la représentation avec le cercle trigonométrique.

# Challenge 2 : Attaque de la femme du milieu (version quantique)

Vous connaissez tous l'attaque de *Man In The Middle* classique, un attaquant intercepte une communication entre deux entités qui ne se rendent pas compte que leur conversation est écoutée. Je vous propose aujourd'hui de découvrir la **version quantique !** **Alice** et **Bob** (oui, encore eux, même les physiciens sont fan), décident de s'échanger une clef secrète avec le **protocole BB84**. Étant certains de la fiabilité de leur protocole, ils laissent une tolérance au bruit. Cependant, **Eve** réussit à intercepter le canal de discussion. Va-t-elle réussir à passer inaperçu ? 

![table_cat_meme.jpg](https://i.imgflip.com/8nfe5l.jpg)

## Protocol BB84 : Utilisation normale
Le protocol BB84 permet de s'échanger une clef de $n$ bits avec $4n$ qubits. Il se base sur le principe que, si quelqu'un venait à écouter la conversation, cette personne serait obligée de **mesurer** les qubits, et donc d'effondrer les fonctions d'ondes associées (une mesure est une projection et donc une perte d'information). **Cet effondrement est ainsi visible**, apportant une erreur de $25\%$ lors de l'étape de vérification. 

### Étape 1 : Alice prépare ses qubits à envoyer
**1.a)** Il faut tout d'abord une clef, pour cela, une chaîne binaire aléatoire de taille $4n$ est générée. 

*Il est possible de générer cette chaîne avec un ordinateur quantique pour de l'aléatoire de meilleure qualité.*

In [ ]:
N = 100
bits_alice = np.random.randint(low=0, high=2, size=(4 * N,))

**1.b)** Alice va ensuite choisir $4n$ bases aléatoires dans $\{+, \times\}$, dans lesquelles elle va encoder ses qubits. *(Veuillez m'excuser pour la confusion entre la base $+$ et l'état $|+\rangle$ qui n'ont rien à voir. La notation + est en fait une croix droite qui correspond aux états en bleu ci-dessous et la croix tournée x aux états en rouge)*.

La base $+$ est la base canonique : 
$$
+ = \left\{ \begin{pmatrix} 1 \\ 0 \end{pmatrix}, \begin{pmatrix} 0 \\ 1 \end{pmatrix}\right\}
$$

Tandis que la base $\times$ est définie par : 
$$
\times = \left\{\frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ 1 \end{pmatrix}, \frac{1}{\sqrt{2}}\begin{pmatrix} -1 \\ 1 \end{pmatrix}\right\}
$$

On peut les visualiser : 

In [ ]:
plot_trig([0, pi/2, pi/4, 3*pi/4], ["blue", "blue", "red", "red"], ["0 (+)", "1 (+)", "0 (x)", "1 (x)"])

L'encodage est entièrement représenté sur ce schéma : $1$ dans la base $\times$ sera codé :
$$
-\frac{\sqrt{2}}{2} \begin{pmatrix} 1 \\ 0 \end{pmatrix} + \frac{\sqrt{2}}{2} \begin{pmatrix} 0 \\ 1 \end{pmatrix} =  -\frac{1}{\sqrt{2}} |0\rangle + \frac{1}{\sqrt{2}} |1\rangle
$$

*Pour rappel, avec l'encodage par rail que l'on a vu précédemment, cela donne :* 
$$
-\frac{1}{\sqrt{2}} |1, 0\rangle\rangle + \frac{1}{\sqrt{2}} |0, 1\rangle\rangle
$$
*On utilise cette notation pour les états de Fock (les rails), pour éviter de confondre avec l'état multiple $|10\rangle$ que l'on découvrira par la suite.*
 
*Peu importe le nom que l'on donne à nos vecteurs de notre base, ici canonique, cela ne change évidemment rien aux calculs. Je ne l'ai pas précisé, mais (presque) tout ce qui est fait dans le cadre de cette compétition est reproduisable sur des ordinateurs quantiques possédants des technologies différentes. Il suffit d'adapter l'encodage du qubit, les calculs sont les mêmes.*

In [ ]:
bases_alice = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4 * N,))])

**1.c)** Alice peut maintenant encoder ses $4n$ qubits dans ses $4n$ bases. *J'utilise des `StateVector`, qui est une généralisation des `BasicState` qu'on a rencontré. Cela me permet d'effectuer des opérations sur les états plus facilement, sans me soucier de la normalisation qui est effectuée automatiquement.* 

***>> Attention aux `StateVector` qui peuvent être méchants, `3 * a + b` marche, alors que `a + 3 * b` non !***
*Attention aussi à la confusion entre le `Statevector` de Qiskit et le `StateVector` de Perceval, on utilise les deux ...*

In [ ]:
qubits_alice = []

# Pour pouvoir tester plus facilement, je définis les valeurs de 0 et de 1 dans la base X, attention, ce 
# ne sont plus des BasicState, mais des StateVector, il faut donc utiliser la fonction measure (voir
# ci-dessous) pour les manipuler
qubits["0x"] = qubits["0"] + qubits["1"] 
qubits["1x"] = qubits["1"] - qubits["0"] 
print(type(qubits["0"]), type(qubits["0x"]))

for bit, basis  in zip(bits_alice, bases_alice):
    if basis == "+":
        s = pcvl.StateVector(qubits["0"]) if bit == 0 else pcvl.StateVector(qubits["1"])
    else: 
        s = qubits["0x"] if bit == 0 else qubits["1x"]
    qubits_alice.append(s)

    # On affiche les 9 premiers pour vérifier :
    if len(qubits_alice) < 10: 
        print(f"Bit à encoder : {bit}, base choisie : {basis}, qubit correspondant : {s}")

Maintenant qu'Alice a préparé ses qubits, elle peut les envoyer à Bob. *Par exemple à l'aide d'un photon.*

### Étape 2 : récupération par Bob et décodage

Bob **ne connait pas les bases dans lesquelles ont été encodés les qubits**, il va donc choisir aléatoirement $4n$ bases comme Alice, et va mesurer les qubits d'Alice. Seuls la moitié des bits seront ainsi juste, étant donné qu'il a une chance sur deux à chaque fois de tomber sur la même base qu'Alice.

Une fois qu'il a mesuré les photons, ceux-ci se sont effondrés, personne ne peut plus les avoir. C'est une notion importante à comprendre. Dans le monde classique, on peut envoyer un message tout en gardant une version de son contenu de notre côté, ce n'est pas comme si à chaque message que l'on envoyait, il s'effaçait de notre téléphone, et de notre mémoire. Dans le monde quantique cependant, c'est le cas ! Il y a même un théorème dédié : le **théorème de non-clonage.** On ne peut pas prendre un état quantique arbitraire inconnu et avoir un appareil qui nous renvoie deux copies de cet état. C'est évidemment très pratique dans le monde de la cyber. On peut effectuer des actions après mesure avec des garanties. Le RGPD quantique déjà au taquet !

**2.a)** Bob choisit aléatoirement $4n$ bases : 

In [ ]:
bases_bob = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4*N,))])

Pour simuler une mesure, on va utiliser le processeur "SLOS" : 

In [ ]:
def measure(input_state, circuit, full=False):
    p = pcvl.Processor("SLOS", circuit)
    p.with_input(input_state)
    sampler = pcvl.algorithm.Sampler(p)

    # Mesure (complète) faite avec 1000 essais, on se retrouve donc avec un résultat semblable 
    # à l'Analyser
    if full:
        sample_count = sampler.sample_count(1000)
        return sample_count['results']
        
    sample_count = sampler.sample_count(1)
    return list(sample_count['results'].keys())[0]

**2.b)** Bob prépare les circuits nécessaires à ces mesures.

Pour mesurer dans la base $+$, il suffit de prendre l'identité vu que c'est notre base canonique : 

In [ ]:
base_p = Circuit(2)

On peut vérifier :

In [ ]:
print(f"""
0 dans la base + : {measure(qubits["0"], base_p, full=True)}
1 dans la base + : {measure(qubits["1"], base_p, full=True)}
0 dans la base x ({qubits["0x"]}) mesurée dans la base + : {measure(qubits["0x"], base_p, full=True)}
1 dans la base x ({qubits["1x"]}) mesurée dans la base + : {measure(qubits["1x"], base_p, full=True)}
""")

Comme prévu, aucun moyen de déterminer le bit d'Alice si Bob se trompe de base. On fait de même pour la base $\times$. Cette fois, on utilise une rotation de $-\frac{\pi}{4}$ (passage de la base $\times$ à notre base canonique). J'utilise pour cela une rotation $y$ avec un angle de $-\frac{\pi}{2}$, cela est dû aux différences de conventions entre la sphère de Bloch et notre cercle trigonométrique. 

In [ ]:
base_x = y_rot(-pi/2)

print(f"""
0 dans la base x : {measure(qubits["0x"], base_x, full=True)}
1 dans la base x : {measure(qubits["1x"], base_x, full=True)}
0 dans la base + ({qubits["0"]}) mesurée dans la base x : {measure(qubits["0"], base_x, full=True)}
1 dans la base + ({qubits["1"]}) mesurée dans la base x : {measure(qubits["1"], base_x, full=True)}
""")

**2.c)** Bob mesure les qubits envoyés par Alice dans ses bases. 

In [ ]:
bits_bob = []
for q, b in zip(qubits_alice, bases_bob): 
    if b == "+":
        bits_bob.append(0 if measure(q, base_p) == qubits["0"] else 1)
    else:
        bits_bob.append(0 if measure(q, base_x) == qubits["0"] else 1)
bits_bob = np.array(bits_bob)

On peut calculer la correspondance avec la clef d'Alice : 

In [ ]:
correspondance_secret_key_bits_bob = bits_bob == bits_alice
np.sum(correspondance_secret_key_bits_bob) / (4 * N)

On a bien $\sim 75\%$ d'équivalence : dans $50\%$ des cas, Alice et Bob ont la même base donc les mêmes valeurs. Il y a $25\%$ en plus, car quand ils n'ont pas la même base, il y a $50\%$ de réussite. 

### Étape 3 : Alice et Bob se mettent d'accord.

Une fois que les qubits ont été mesurés par Bob, Bob va **rendre ses bases publiques**, et Alice va pouvoir lui dire où il y a correspondance, et donc où ils peuvent être sûr d'avoir la même chose.

In [ ]:
correspondance_bases_alice_bob = bases_bob == bases_alice 
half_bits_bob = bits_bob[correspondance_bases_alice_bob]
half_bits_alice = bits_alice[correspondance_bases_alice_bob]
# ATTENTION : Ne pas relancer la cellule toute seule, relancer tout le notebook pour rafraichir cette cellule correctement. 

### Étape 4 : Vérification de l'intégrité de la communication.

Pour vérifier qu'il n'y a pas eu d'écoute, Alice et Bob vont **rendre publique la moitié de leur clef commune.** Si il y a correspondance totale, alors on est sûr que personne n'a écouté. 

In [ ]:
last_slice = len(half_bits_bob) // 2
verification = half_bits_bob[:last_slice] == half_bits_alice[:last_slice]
print(f"Pourcentage de correspondance : {int(np.sum(verification) / last_slice * 100)}%")

D'où *in fine*, la clef secrète avec une taille environ égale à $n$ (On peut trouver un $n'$ tel que la probabilité d'avoir $|clef| > n$ soit arbitrairement grande, puis couper à $n$) : 

In [ ]:
secret_key = half_bits_bob[last_slice:]
print(f"Secret key : {secret_key}, taille : {len(secret_key)}")

## Protocol BB84 : Ève entre en scène 

Tout est identique, sauf que maintenant, Ève intercepte la communication et va essayer de déduire une partie de la clef échangée. 

### Étape 1 : Réimplémentez le protocole, en rajoutant Ève au milieu *(cette étape n'est pas nécessaire pour obtenir le drapeau, mais recommandé pour faciliter la suite)*

Ève utilise dans un premier temps la même stratégie que Bob et Alice, elle se fait d'abord passer pour Bob, puis pour Alice, en effectuant les mêmes actions d'encodage/ décodage. 

In [ ]:
N = 100

# Alice prépare ses qubits
bits_alice = np.random.randint(low=0, high=2, size=(4 * N,))
bases_alice = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4 * N,))])
qubits_alice = []
for bit, basis  in zip(bits_alice, bases_alice):
    if basis == "+":
        s = pcvl.StateVector(qubits["0"]) if bit == 0 else pcvl.StateVector(qubits["1"])
    else:
        s = qubits["0x"] if bit == 0 else qubits["1x"]
    qubits_alice.append(s)


# Ève les intercepte et applique la même méthode que Bob en se faisant passer pour lui
bits_eve = []
raise NotImplementedError


bits_eve = np.array(bits_eve)

# Elle renvoie ensuite les qubits correspondants pour se faire passer pour Alice
qubits_eve = []
raise NotImplementedError
    

# Bob reçoit les qubits d'Ève et applique la même méthode que précédemment 
bases_bob = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4 * N,))])
bits_bob = []
for q, b in zip(qubits_eve, bases_bob):
    if b == "+":
        bits_bob.append(0 if measure(q, base_p) == qubits["0"] else 1)
    else:
        bits_bob.append(0 if measure(q, base_x) == qubits["0"] else 1)
bits_bob = np.array(bits_bob)


# Dernière étape : mise en commun
correspondance_bases_alice_bob = bases_bob == bases_alice
half_bits_alice = bits_alice[correspondance_bases_alice_bob]
half_bits_bob = bits_bob[correspondance_bases_alice_bob]
last_slice = len(half_bits_alice) // 2


# Vérification du bon déroulé 
verification = half_bits_alice[:last_slice] == half_bits_bob[:last_slice]
correspondance_percentage = int(np.sum(verification) / last_slice * 100)

print(f"Pourcentage de correspondance : {correspondance_percentage}%, d'où une erreur de {100 - correspondance_percentage}%.")

secret_key_step_1 = half_bits_bob[:last_slice]

En faisant les calculs, on montre que l'erreur est en moyenne de $25\%$· En échangeant un grand nombre de bits, on se rapproche de cette moyenne et l'on peut donc détecter une écoute. 

In [ ]:
print(secret_key_step_1)

Ce n'est pas exactement la clef, il faudra encore utiliser des codes correction d'erreurs pour qu'Alice et Bob se retrouvent avec la même clef. Dans le cadre de ce challenge, on va s'arrêter ici.

Dans ce cas-là, Ève se fait détecter. Mais il est possible de réduire l'erreur, certes de peu, mais cela peut permettre de passer pour du bruit. On va supposer pour simplifier que : **une erreur en dessous de $20\%$ est considérée comme du bruit.**

### Étape 2 : Ève contre-attaque, trouvez un moyen de faire descendre l'erreur moyenne à $20\%$ (ou en dessous).

*Le protocole reste le même, les seuls paramètres modifiables sont les bases de mesure pour mesurer les qubits d'Alice et les qubits à envoyer à Bob. Le pourcentage de correspondance doit être au-dessus de $79$ pour $N = 5000$ pour passer les tests.*

In [ ]:
N = 100

# Alice prépare ses qubits
bits_alice = np.random.randint(low=0, high=2, size=(4 * N,))
bases_alice = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4 * N,))])
qubits_alice = []
for bit, basis  in zip(bits_alice, bases_alice):
    if basis == "+":
        s = pcvl.StateVector(qubits["0"]) if bit == 0 else pcvl.StateVector(qubits["1"])
    else:
        s = qubits["0x"] if bit == 0 else qubits["1x"]
    qubits_alice.append(s)


# Interception
bits_eve = ...
raise NotImplementedError

# Envoi
qubits_eve = []
raise NotImplementedError 

# Bob reçoit les qubits d'Ève et applique la même méthode que précédemment 
bases_bob = np.array(["+" if b == 0 else "x" for b in np.random.randint(low=0, high=2, size=(4 * N,))])
bits_bob = []
for q, b in zip(qubits_eve, bases_bob):
    if b == "+":
        bits_bob.append(0 if measure(q, base_p) == qubits["0"] else 1)
    else:
        bits_bob.append(0 if measure(q, base_x) == qubits["0"] else 1)
bits_bob = np.array(bits_bob)


# Dernière étape : mise en commun
correspondance_bases_alice_bob = bases_bob == bases_alice
half_bits_alice = bits_alice[correspondance_bases_alice_bob]
half_bits_bob = bits_bob[correspondance_bases_alice_bob]
last_slice = len(half_bits_alice) // 2


# Vérification du bon déroulé 
verification = half_bits_alice[:last_slice] == half_bits_bob[:last_slice]
correspondance_percentage = int(np.sum(verification) / last_slice * 100)

print(f"Pourcentage de correspondance : {correspondance_percentage}%, d'où une erreur de {100 - correspondance_percentage}%.")

secret_key_step_2 = half_bits_bob[:last_slice]

## Récupération du drapeau 

Le jour où j'arrive à faire marcher `pcvl.random_seed(66)` je vous demanderai les clefs. En attendant, je veux bien les deux bases utilisées lors de l'étape 2, ainsi que les deux types de qubits envoyés par Ève. 

Vous pouvez venir en DM si vous avez des problèmes. 

In [ ]:
def circuit_to_list(circuit: Circuit) -> List[List[Tuple[float, float]]]:
    return [[(x.real, x.imag) for x in l] for l in np.array(circuit.compute_unitary())]
    
def state_vector_to_list(sv: StateVector) -> List[Tuple[float, float]]:
    if type(sv) is not StateVector:
        sv = pcvl.StateVector(sv)
    sv.normalize()
    r = [(0., 0.), (0., 0.)]
    for k, v in sv:
        r[int(qubits_[k])] = (v.real, v.imag)
    return r

def list_to_state_vector(p: List[Tuple[float, float]]) -> StateVector:
    return complex(p[0][0], p[0][1]) * StateVector([1, 0]) + complex(p[1][0], p[1][1]) * StateVector([0, 1]) 

In [ ]:
# Exemple pour la version de l'étape 1 : 
# example_base_eve_1 = base_p 
# example_base_eve_2 = base_x
# example_qubit_eve_1 = qubits["0"]
# example_qubit_eve_2 = qubits["1"]
# example_qubit_eve_3 = qubits["0x"]
# example_qubit_eve_4 = qubits["1x"]

d = {
    "base_eve_1": circuit_to_list(...),
    "base_eve_2": circuit_to_list(...),
    "qubit_eve_1": state_vector_to_list(...),
    "qubit_eve_2": state_vector_to_list(...),
    "qubit_eve_3": state_vector_to_list(...),
    "qubit_eve_4": state_vector_to_list(...)
}
raise NotImplementedError

In [ ]:
import requests as rq

URL = ...
# URL = "https://perceval.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

In [ ]:
rq.post(URL + "/challenges/2", json=d).json()